In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from scipy.sparse import csr_matrix
import random as rnd
import pickle
import sys


from datetime import datetime


from sklearn import preprocessing
from sklearn.model_selection import train_test_split




In [2]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
def generate_random_array(median,std,size):
    output=[0]*size    
    for index in range(0,size//2):
        random_nr=rnd.uniform(-std,std)
        output[2*index]=median+random_nr
        output[2*index+1]=median-random_nr
    if(size % 2 != 0):
        output[size-1]=rnd.uniform(median-std,median+std)
    return output



In [ ]:
def class_selection(x):
  if 1 in set(x) and 2 in set(x):
    return 3
  if 1 in set(x):
    return 1
  if 2 in set(x):
    return 2
  if 3 in set(x):
    return 3
  else:
    return 0

In [ ]:
def label_attacks(x):
  if x != 'BENIGN' and x != 'DDoS':
    return 'DoS'
  return x

In [ ]:
traffic = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TrafficLabelling/combined.csv")
print(f'Number of Rows: {traffic.shape[0]}')
print(f'Number of Columns: {traffic.shape[1]}')
pd.set_option('display.max_colwidth', None)
traffic.head()

Number of Rows: 2660377
Number of Columns: 85


,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,...,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165,443,192.168.10.5,54865,6,7/7/2017 3:30,3,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,192.168.10.5-104.16.28.216-55054-80-6,104.16.28.216,80,192.168.10.5,55054,6,7/7/2017 3:30,109,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,1.100917e+05,18348.62385,109.0,0.0,109.0,109.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,192.168.10.5-104.16.28.216-55055-80-6,104.16.28.216,80,192.168.10.5,55055,6,7/7/2017 3:30,52,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,2.307692e+05,38461.53846,52.0,0.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,192.168.10.16-104.17.241.25-46236-443-6,104.17.241.25,443,192.168.10.16,46236,6,7/7/2017 3:30,34,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,3.529412e+05,58823.52941,34.0,0.0,34.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,31,329,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,192.168.10.5-104.19.196.102-54863-443-6,104.19.196.102,443,192.168.10.5,54863,6,7/7/2017 3:30,3,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,32,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [ ]:
# Remove duplicate entries
traffic.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)

# reformat Timestamp to Unix time
# traffic[' Timestamp'] = pd.to_datetime(traffic[' Timestamp'])
# traffic[' Timestamp']= traffic[' Timestamp'].values.astype(np.int64) // 10 ** 9

# remove Timestamp column
traffic = traffic.drop(columns=[' Timestamp'])

# Remove constant columns
traffic = traffic.loc[:, traffic.apply(pd.Series.nunique) != 1]




KeyError: ignored

In [ ]:
# Remove column with NaN or Inf
traffic = traffic[~traffic.isin([np.nan, np.inf, -np.inf]).any(1)]

# remove Flow ID4/1AY0e-g6XuChtvXVE6YBzBd8J_mRl94t6Rh5jd18f3EvtJaQd0f_H8wY4Bss
traffic = traffic.drop(columns=['Flow ID'])



In [ ]:
raw_labels = set(list(traffic[' Label']))

traffic = traffic[traffic[' Label']  != "SSH-Patator"]
traffic = traffic[traffic[' Label']  != "PortScan"]
traffic = traffic[traffic[' Label']  != "Heartbleed"]
traffic = traffic[traffic[' Label']  != "FTP-Patator"]
traffic = traffic[traffic[' Label']  != "Infiltration"]
traffic = traffic[traffic[' Label']  != "Bot"]
raw_labels

{'BENIGN',
 'Bot',
 'DDoS',
 'DoS GoldenEye',
 'DoS Hulk',
 'DoS Slowhttptest',
 'DoS slowloris',
 'FTP-Patator',
 'Heartbleed',
 'Infiltration',
 'PortScan',
 'SSH-Patator'}

In [ ]:

traffic[' Label'] = traffic[' Label'].apply(label_attacks)


raw_labels = list(traffic[' Label'])

for raw_label in set(raw_labels):
  print(str(raw_label) + ": " + str(raw_labels.count(raw_label)))

traffic[' Label'] = traffic[' Label'].astype('category')
traffic[' Label'] = traffic[' Label'].cat.codes

traffic[' Label']


BENIGN: 2098517
DDoS: 128025
DoS: 244124


0          0
1          0
2          0
3          0
4          0
          ..
2660372    0
2660373    0
2660374    0
2660375    0
2660376    0
Name:  Label, Length: 2470666, dtype: int8

In [ ]:
# combine ' Source IP' and ' Source Port'
traffic[' Source IP'] = traffic[' Source IP'] + ':' + traffic[' Source Port'].astype(str)
traffic[' Destination IP'] = traffic[' Destination IP'] + ':' + traffic[' Destination Port'].astype(str)
traffic.rename({' Source IP': ' SrcHost', ' Destination IP': ' DstHost'}, axis=1, inplace=True)

traffic = traffic.drop(columns=[' Source Port'])
traffic = traffic.drop(columns=[' Destination Port'])

traffic.head()


,SrcHost,DstHost,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,104.16.207.165:443,192.168.10.5:54865,6,3,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,40,0,666666.666700,0.000000,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,9.0,6.0,0.0,40,2,12,0,0,33,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,104.16.28.216:80,192.168.10.5:55054,6,109,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,1.100917e+05,18348.62385,109.0,0.0,109.0,109.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,9174.311927,9174.311927,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,104.16.28.216:80,192.168.10.5:55055,6,52,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,2.307692e+05,38461.53846,52.0,0.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,19230.769230,19230.769230,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,104.17.241.25:443,192.168.10.16:46236,6,34,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,3.529412e+05,58823.52941,34.0,0.0,34.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,29411.764710,29411.764710,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,1,6,1,6,31,329,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,104.19.196.102:443,192.168.10.5:54863,6,3,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,40,0,666666.666700,0.000000,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,9.0,6.0,0.0,40,2,12,0,0,32,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
columns = list(traffic.columns)


In [ ]:
# create host dict {host: node_id}
src_hosts = set(traffic[' SrcHost'])
dst_hosts = set(traffic[' DstHost'])
hosts = list(src_hosts.union(dst_hosts))

                
host_dict = {hosts[i]: i for i in range(len(hosts))}   # dictionary of ip addresses { ip_addr: node_id }
print("Number of nodes: " + str(len(host_dict)))

Number of nodes: 435766


In [ ]:
# adding a group_size column to the original dataframe
traffic['group_size'] = traffic.groupby([' SrcHost'])[' DstHost'].transform('size')


In [ ]:
# define aggregation functions for all columns
aggregations = {col_name: "mean" for col_name in columns}
aggregations.pop(' Protocol', None)
aggregations.pop(' SrcHost', None)
aggregations.pop(' DstHost', None)
aggregations.pop(' Label', None)

aggregations.update({' Label': lambda x : class_selection(x)})
print(aggregations)


{' Flow Duration': 'mean', ' Total Fwd Packets': 'mean', ' Total Backward Packets': 'mean', 'Total Length of Fwd Packets': 'mean', ' Total Length of Bwd Packets': 'mean', ' Fwd Packet Length Max': 'mean', ' Fwd Packet Length Min': 'mean', ' Fwd Packet Length Mean': 'mean', ' Fwd Packet Length Std': 'mean', 'Bwd Packet Length Max': 'mean', ' Bwd Packet Length Min': 'mean', ' Bwd Packet Length Mean': 'mean', ' Bwd Packet Length Std': 'mean', 'Flow Bytes/s': 'mean', ' Flow Packets/s': 'mean', ' Flow IAT Mean': 'mean', ' Flow IAT Std': 'mean', ' Flow IAT Max': 'mean', ' Flow IAT Min': 'mean', 'Fwd IAT Total': 'mean', ' Fwd IAT Mean': 'mean', ' Fwd IAT Std': 'mean', ' Fwd IAT Max': 'mean', ' Fwd IAT Min': 'mean', 'Bwd IAT Total': 'mean', ' Bwd IAT Mean': 'mean', ' Bwd IAT Std': 'mean', ' Bwd IAT Max': 'mean', ' Bwd IAT Min': 'mean', 'Fwd PSH Flags': 'mean', ' Fwd URG Flags': 'mean', ' Fwd Header Length': 'mean', ' Bwd Header Length': 'mean', 'Fwd Packets/s': 'mean', ' Bwd Packets/s': 'mean'

In [ ]:
# low activity grouping
low_activities = traffic[traffic['group_size'] == 1]
low_activities = low_activities[low_activities[' Label'] == 0]
low_activities = low_activities.groupby(' SrcHost', as_index=False).agg(aggregations)
low_activities



,SrcHost,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.0.0.0:0,36685713,2,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.054517,36700000.0,0.000000,36700000.0,36700000.0,36700000.0,36700000.0,0.000000,36700000.0,36700000.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.054517,0.000000,0.0,0.0,0.0,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,2,0,0,0,-1,-1,0,0,0.0,0.0,0.0,0.0,36700000.0,0.0,36700000.0,36700000.0,0
1,1.193.219.21:443,648,3,0,258.0,0.0,258.0,0.0,86.0,148.956369,0.0,0.0,0.0,0.0,3.981481e+05,4629.629630,324.0,452.548340,644.0,4.0,648.0,324.0,452.548340,644.0,4.0,0.0,0.0,0.0,0.0,0.0,0,0,96,0,4629.629630,0.000000,0.0,258.0,64.5,129.000000,16641.0,0,0,0,0,1,0,0,0,0.0,86.0,86.0,0.0,96,3,258,0,0,61,-1,1,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1.234.58.131:80,186262,2,0,6.0,0.0,6.0,0.0,3.0,4.242641,0.0,0.0,0.0,0.0,3.221269e+01,10.737563,186262.0,0.000000,186262.0,186262.0,186262.0,186262.0,0.000000,186262.0,186262.0,0.0,0.0,0.0,0.0,0.0,0,0,52,0,10.737563,0.000000,0.0,6.0,2.0,3.464102,12.0,0,0,0,0,1,0,0,0,0.0,3.0,3.0,0.0,52,2,6,0,0,140,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1.9.56.32:80,83,1,1,6.0,6.0,6.0,6.0,6.0,0.000000,6.0,6.0,6.0,0.0,1.445783e+05,24096.385542,83.0,0.000000,83.0,83.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,12048.192771,12048.192771,6.0,6.0,6.0,0.000000,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,1,6,1,6,980,253,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,101.102.235.200:443,2,1,1,6.0,6.0,6.0,6.0,6.0,0.000000,6.0,6.0,6.0,0.0,6.000000e+06,1000000.000000,2.0,0.000000,2.0,2.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,500000.000000,500000.000000,6.0,6.0,6.0,0.000000,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,1,6,1,6,279,251,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121451,98.158.54.101:443,67580,2,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,29.594555,67580.0,0.000000,67580.0,67580.0,67580.0,67580.0,0.000000,67580.0,67580.0,0.0,0.0,0.0,0.0,0.0,0,0,64,0,29.594555,0.000000,0.0,0.0,0.0,0.000000,0.0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,64,2,0,0,0,62512,-1,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
121452,98.158.54.103:443,119023,2,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,16.803475,119023.0,0.000000,119023.0,119023.0,119023.0,119023.0,0.000000,119023.0,119023.0,0.0,0.0,0.0,0.0,0.0,0,0,64,0,16.803475,0.000000,0.0,0.0,0.0,0.000000,0.0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,64,2,0,0,0,62512,-1,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
121453,98.158.54.104:443,2110,3,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,1421.800948,1055.0,1429.769912,2066.0,44.0,2110.0,1055.0,1429.769912,2066.0,44.0,0.0,0.0,0.0,

In [ ]:
# attackers grouping
atk_activities = traffic[traffic[' Label'] != 0]
atk_activities = atk_activities.groupby(' SrcHost', as_index=False).agg(aggregations)
print("Number of DDoS atkers: ", str(list(atk_activities[' Label']).count(1)))
print("Number of DoS atkers: ", str(list(atk_activities[' Label']).count(2)))
print("number of atker both ddos and dos: "  + str(list(atk_activities[' Label']).count(3)))
atk_activities




Number of DDoS atkers:  34276
Number of DoS atkers:  3001
number of atker both ddos and dos: 11117


,SrcHost,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,172.16.0.1:14099,4.025588e+06,4.000000,3.000000,28.000000,5800.50,13.000000,3.0,7.333333,5.131601,2190.000000,0.0,966.750,878.894974,510405.728721,395.394360,1.005685e+06,2.011044e+06,4.023958e+06,11.500000,4.025022e+06,1.007538e+06,2.012823e+06,4.023968e+06,66.00,5.685500e+03,1.137100e+03,2.289932e+03,5.228000e+03,25.000000,0.0,0,86.000000,66.000000,132.005422,263.388938,3.0,2193.000000,584.350000,822.620881,1.353410e+06,0.0,0.0,0,0.500000,0.500000,0.0,0,0,1.000000,649.544445,7.333333,966.750,86.000000,4.000000,28.000000,3.000000,5800.50,4224.000000,114.000000,3.000000,20.0,990.500000,0.0,990.500000,990.500000,4.018902e+06,0.0,4.018902e+06,4.018902e+06,1
1,172.16.0.1:14100,4.024634e+06,4.000000,3.000000,28.000000,5800.50,13.000000,3.0,7.333333,5.131601,2190.000000,0.0,966.750,878.894974,506889.999183,392.673050,1.005442e+06,2.010842e+06,4.023367e+06,3.000000,4.023930e+06,1.007240e+06,2.012671e+06,4.023428e+06,3.00,5.673500e+03,1.134700e+03,2.188398e+03,5.032000e+03,8.500000,0.0,0,86.000000,66.000000,131.098370,261.574680,3.0,2193.000000,584.350000,822.620881,1.353410e+06,0.0,0.0,0,0.500000,0.500000,0.0,0,0,1.000000,649.544445,7.333333,966.750,86.000000,4.000000,28.000000,3.000000,5800.50,4224.000000,114.000000,3.000000,20.0,499.500000,0.0,499.500000,499.500000,4.018400e+06,0.0,4.018400e+06,4.018400e+06,1
2,172.16.0.1:14101,4.023676e+06,4.000000,2.000000,28.000000,5800.50,13.000000,3.0,7.333333,5.131601,5067.500000,0.0,1450.125,2435.912915,558187.175882,336.364832,1.005485e+06,2.010576e+06,4.022238e+06,32.000000,4.022979e+06,1.006872e+06,2.011792e+06,4.022269e+06,212.00,5.176000e+03,1.725333e+03,2.476014e+03,4.574500e+03,95.000000,0.0,0,86.000000,46.000000,144.334107,192.030725,3.0,5070.500000,729.687500,1772.326013,6.300000e+06,0.0,0.0,0,0.500000,0.500000,0.0,0,0,0.500000,834.100000,7.333333,1450.125,86.000000,4.000000,28.000000,2.000000,5800.50,4224.000000,114.000000,3.000000,20.0,498.500000,0.0,498.500000,498.500000,4.017970e+06,0.0,4.017970e+06,4.017970e+06,1
3,172.16.0.1:14102,4.022290e+06,4.000000,2.500000,28.000000,5800.50,13.000000,3.0,7.333333,5.131601,3650.000000,0.0,1160.100,1511.254049,591586.277390,407.352972,1.005046e+06,2.009942e+06,4.021044e+06,2.500000,4.021379e+06,1.006345e+06,2.011310e+06,4.021068e+06,2.50,4.890000e+03,1.222500e+03,1.917825e+03,4.075000e+03,95.000000,0.0,0,86.000000,56.000000,152.951832,254.401140,3.0,3653.000000,648.944445,1230.326493,3.027407e+06,0.0,0.0,0,0.500000,0.500000,0.0,0,0,0.500000,730.287500,7.333333,1160.100,86.000000,4.000000,28.000000,2.500000,5800.50,4224.000000,114.000000,3.000000,20.0,309.000000,0.0,309.000000,309.000000,4.017068e+06,0.0,4.017068e+06,4.017068e+06,1
4,172.16.0.1:14103,4.021305e+06,4.000000,2.000000,28.000000,5800.50,13.000000,3.0,7.333333,5.131601,4380.000000,0.0,1450.125,2064.159651,659727.241612,397.496841,1.004959e+06

In [ ]:
basic_flows = traffic.groupby([' SrcHost', ' DstHost'], as_index=False)

basic_flows = basic_flows.agg(aggregations)
basic_flows
print(set(basic_flows[' Label']))

non_atkers = basic_flows.groupby([' Label']).get_group(0)
dos_atkers = basic_flows.groupby([' Label']).get_group(1)
ddos_atkers = basic_flows.groupby([' Label']).get_group(2)
both_atkers = basic_flows.groupby([' Label']).get_group(3)

print("Number of non_attackers: " + str(len(non_atkers)))
print("Number of ddos " + str(len(dos_atkers)))
print("Number of dos " + str(len(ddos_atkers)))
print("Number of both " + str(len(both_atkers)))
basic_flows


{0, 1, 2, 3}
Number of non_attackers: 1167483
Number of ddos 34278
Number of dos 3001
Number of both 11117


,SrcHost,DstHost,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.0.0.0:0,224.0.0.22:0,36685713.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.054517,36700000.0,0.0000,36700000.0,36700000.0,36700000.0,36700000.0,0.0,36700000.0,36700000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054517,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,36700000.0,0.0,36700000.0,36700000.0,0
1,1.1.70.73:80,192.168.10.15:52772,344.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,34883.720930,5813.953488,344.0,0.0000,344.0,344.0,344.0,344.0,0.0,344.0,344.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,5813.953488,0.000000,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,6.0,0.0,40.0,2.0,12.0,0.0,0.0,4380.0,-1.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1.1.70.73:80,192.168.10.17:48301,315.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,6349.206349,315.0,0.0000,315.0,315.0,315.0,315.0,0.0,315.0,315.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,6349.206349,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,2.0,0.0,0.0,0.0,4380.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1.1.70.73:80,192.168.10.19:35264,179391.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,16.723247,89695.5,126775.0535,179339.0,52.0,179391.0,179391.0,0.0,179391.0,179391.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,32.0,11.148831,5.574416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,2.0,0.0,1.0,0.0,4677.0,240.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1.1.70.73:80,192.168.10.19:35275,417.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,4796.163070,417.0,0.0000,417.0,417.0,417.0,417.0,0.0,417.0,417.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,4796.163070,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,2.0,0.0,0.0,0.0,4380.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215874,99.192.248.32:80,192.168.10.5:49372,55.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,218181.818182,36363.636364,55.0,0.0000,55.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,18181.818182,18181.818182,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,6.0,6.0,20.0,1.0,6.0,1.0,6.0,46.0,256.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1215875,99.192.248.32:80,192.168.10.5:49419,45.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,266666.666667,44444.444444,45.0,0.0000,45.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,22222.222222,22222.222222,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.

In [ ]:
#activity_flows
activity_flows = basic_flows.groupby([' SrcHost'], as_index=False)
activity_flows = activity_flows.agg(aggregations)

print(set(activity_flows[' Label']))
non_atkers = activity_flows.groupby([' Label']).get_group(0)
ddos_atkers = activity_flows.groupby([' Label']).get_group(1)
dos_atkers = activity_flows.groupby([' Label']).get_group(2)
both_atkers = activity_flows.groupby([' Label']).get_group(3)

print(activity_flows)
print("Number of non-atkers: " + str(len(non_atkers)))
print("Number of ddos " + str(len(ddos_atkers)))
print("Number of dos " + str(len(dos_atkers)))
print("Number of both " + str(len(both_atkers)))


{0, 1, 2, 3}
                  SrcHost   Flow Duration  ...    Idle Min   Label
0               0.0.0.0:0    3.668571e+07  ...  36700000.0       0
1            1.1.70.73:80    4.511675e+04  ...         0.0       0
2        1.193.219.21:443    6.480000e+02  ...         0.0       0
3        1.193.219.24:443    1.038322e+05  ...         0.0       0
4          1.226.51.14:80    4.217540e+04  ...         0.0       0
...                   ...             ...  ...         ...     ...
425112  98.158.58.110:443    4.627100e+04  ...         0.0       0
425113  98.158.62.102:443    4.732533e+04  ...         0.0       0
425114  98.158.62.103:443    1.156000e+03  ...         0.0       0
425115   99.192.248.32:80    8.496417e+03  ...         0.0       0
425116  99.193.229.25:443    3.000000e+00  ...         0.0       0

[425117 rows x 71 columns]
Number of non-atkers: 376723
Number of ddos 34276
Number of dos 3001
Number of both 11117


In [ ]:
# create low profiles dataframe
num_hosts = len(host_dict)
num_src_hosts = len(activity_flows[' SrcHost'])
num_noflow_hosts = num_hosts - num_src_hosts


no_flow_prof_columns = columns[3:-2]
print(no_flow_prof_columns)
num_low_hosts = round(1*num_noflow_hosts)   #### adjust benign/attackers ratio here
low_activities_prof = np.zeros((num_low_hosts, len(no_flow_prof_columns)))
for col, attr in enumerate(no_flow_prof_columns):
  median = low_activities[attr].median()
  std = np.std(low_activities[attr])
  low_activities_prof[:, col] = generate_random_array(median, std, num_low_hosts)


low_profile_df = pd.DataFrame(low_activities_prof, columns = no_flow_prof_columns)
low_profile_df[' Label'] = [0 for i in range(num_low_hosts)]
low_profile_df[' SrcHost'] = [hosts[num_src_hosts + i] for i in range(num_low_hosts)]
low_profile_df



[' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Fwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count', ' ECE Flag Count', ' Down/Up R

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Label,SrcHost
0,1.954492e+07,-7.287604,-5.644695,5586.940388,-9659.212729,161.386698,44.045507,13.646809,-90.946909,-364.033249,156.163746,-24.386704,-174.539376,-1.271502e+06,28835.010172,878528.650261,-1.461770e+06,1.196967e+06,-172927.100820,-2.530678e+07,-8.941424e+05,-1.496573e+06,1.811769e+06,-99332.418747,-1.026393e+07,3.566718e+05,8.026487e+05,-2.437046e+06,26695.023335,-0.035856,-0.001396,-9.639581e+06,-4.583285e+04,-30073.800793,-533.617977,56.208776,71.119735,192.094782,-39.637463,154262.004675,0.045727,0.060954,0.027105,-0.139103,-0.160443,0.124021,0.007735,-0.008935,1.275060,197.339748,99.675044,195.914240,9.604042e+06,2.891007,-1164.907896,29.293648,28429.548808,-1310.095476,-1234.052795,-0.523364,-8.756162e+05,-126303.335430,153606.072283,560691.149033,274939.348068,3.589833e+06,537709.271837,9.373226e+05,0,172.16.0.1:58308
1,-1.949700e+07,11.287604,9.644695,-5442.940388,10051.212729,-89.386698,21.954493,56.353191,90.946909,562.033249,9.836254,218.386704,174.539376,1.299811e+06,-28500.478099,-862605.316927,1.489222e+06,-1.149147e+06,172933.100820,2.530679e+07,8.941484e+05,1.496573e+06,-1.811763e+06,99338.418747,1.026394e+07,-3.566638e+05,-8.026487e+05,2.437054e+06,-26687.023335,0.035856,0.001396,9.639709e+06,4.596085e+04,30241.220850,698.811666,9.791224,128.880265,-64.894782,105.364169,-152102.004675,-0.045727,-0.060954,-0.027105,0.139103,0.160443,-0.124021,-0.007735,0.008935,0.724940,-41.339748,-29.675044,-1.914240,-9.603914e+06,1.108993,1308.907896,-25.293648,-28037.548808,1308.095476,1232.052795,2.523364,8.756802e+05,126303.335430,-153606.072283,-560691.149033,-274939.348068,-3.589833e+06,-537709.271837,-9.373226e+05,0,192.168.10.19:64688
2,-1.480138e+07,1.806469,-25.311987,-7097.827057,-35111.302955,244.816085,93.796300,-7.819202,39.626205,-1.750457,91.446856,173.255627,139.777937,-4.588043e+06,20632.875372,-37131.510763,-1.467457e+06,1.712923e+06,-13371.060036,-2.098363e+07,4.644361e+05,-5.321712e+03,-2.278317e+05,689.539131,-2.197400e+07,1.236920e+06,1.567128e+06,2.822355e+06,-70492.680991,-0.042612,0.005045,-8.154232e+06,2.034291e+06,-62164.086075,34996.367237,27.340278,542.000757,171.208105,162.707258,-17289.254453,0.058273,0.025192,0.038540,-0.010651,0.018701,-0.070362,-0.006526,0.020349,0.954947,202.127157,-30.525482,297.095516,-1.276728e+07,4.899133,-5248.403097,3.191025,-35217.872091,-2645.370666,5907.155356,7.309199,-2.200007e+06,-204929.634549,250779.387429,99023.091790,-92850.720014,-5.242921e+05,489422.008412,7.010134e+04,0,192.168.10.17:53476
3,1.484930e+07,2.193531,29.311987,7241.827057,35503.302955,-172.816085,-27.796300,77.819202,-39.626205,199.750457,74.553144,20.744373,-139.777937,4.616352e+06,-20298.343299,53054.844097,1.494908e+06,-1.665103e+06,13377.060036,2.098363e+07,-4.644301e+05,5.321712e+03,2.278377e+05,-683.539131,2.197401e+07,-1.236912e+06,-1.567128e+06,-2.822347e+06,

In [ ]:
# combines all the dataframes
activity_flows_all = activity_flows.append(low_profile_df, ignore_index=True)



In [ ]:
### output csv file for evaluation with other models
activity_flows_all.to_csv("hosts_multclass1.csv")

In [ ]:
# reformat Source IP
activity_flows_all[" SrcHost"] = activity_flows_all[" SrcHost"].map(host_dict)
print(activity_flows_all)

         SrcHost   Flow Duration  ...    Idle Min   Label
0         162522    3.668571e+07  ...  36700000.0       0
1         406093    4.511675e+04  ...         0.0       0
2         338422    6.480000e+02  ...         0.0       0
3         260827    1.038322e+05  ...         0.0       0
4          11565    4.217540e+04  ...         0.0       0
...          ...             ...  ...         ...     ...
435761    435761    2.301853e+07  ...         NaN       0
435762    435762   -2.297061e+07  ...         NaN       0
435763    435763   -1.694784e+07  ...         NaN       0
435764    435764    1.699576e+07  ...         NaN       0
435765    435765    2.660406e+07  ...         NaN       0

[435766 rows x 71 columns]


In [ ]:
activity_flows_all = activity_flows_all.sort_values(by=' SrcHost', ignore_index=True)
activity_flows_all

,SrcHost,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,3.118950e+04,2.000000,2.000000,83.000000,220.000000,41.500000,41.500000,41.500000,0.000000,110.000000,110.000000,110.000000,0.000000,6.167898e+05,7665.785090,1.039650e+04,1.796298e+04,3.113700e+04,5.500000,5.500000e+00,5.500000e+00,0.000000e+00,5.500000e+00,5.500000,4.700000e+01,4.700000e+01,0.000000e+00,4.700000e+01,47.000000,0.000000,0.000000,5.200000e+01,5.200000e+01,3832.892546,3832.892546,41.500000,110.000000,68.900000,37.518995,1651.350000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,86.125000,41.500000,110.000000,5.200000e+01,2.000000,83.000000,2.000000,220.000000,-1.000000,-1.000000,1.000000,2.600000e+01,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0
1,1,1.665178e+05,11.500000,15.500000,992.500000,17994.000000,681.000000,8.500000,87.102273,194.608058,1271.000000,36.500000,543.401042,392.465823,6.473062e+04,144.226032,8.301943e+03,1.517778e+04,5.191050e+04,6.250000,1.587452e+05,1.128107e+04,1.504742e+04,4.819675e+04,6.250000,1.004762e+05,1.012244e+04,1.028068e+04,3.426425e+04,219.750000,0.000000,0.000000,3.530000e+02,4.850000e+02,71.994511,72.231521,8.500000,1451.750000,355.083803,424.785765,256356.490566,0.000000,0.000000,0.000000,0.750000,0.000000,0.000000,0.000000,0.000000,0.750000,368.930882,87.102273,543.401042,3.530000e+02,11.500000,992.500000,15.500000,17994.000000,49151.000000,134.000000,5.750000,3.200000e+01,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0
2,2,1.720000e+02,2.000000,2.000000,88.000000,188.000000,44.000000,44.000000,44.000000,0.000000,94.000000,94.000000,94.000000,0.000000,1.604651e+06,23255.813950,5.733333e+01,9.237604e+01,1.640000e+02,4.000000,4.000000e+00,4.000000e+00,0.000000e+00,4.000000e+00,4.000000,4.000000e+00,4.000000e+00,0.000000e+00,4.000000e+00,4.000000,0.000000,0.000000,6.400000e+01,6.400000e+01,11627.906980,11627.906980,44.000000,94.000000,64.000000,27.386128,750.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,80.000000,44.000000,94.000000,6.400000e+01,2.000000,88.000000,2.000000,188.000000,-1.000000,-1.000000,1.000000,3.200000e+01,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0
3,3,3.526725e+07,10.611111,8.444444,795.833333,3041.277778,309.833333,0.000000,50.903690,104.922874,809.000000,0.000000,269.197025,332.365205,6.682707e+03,31.021370,1.334022e+06,1.941389e+06,4.305717e+06,33317.833333,3.424697e+07,2.342014e+06,2.061361e+06,4.215422e+06,18969.833333,3.404826e+07,2.464128e+06,1.579986e+06,3.752359e+06,2344.888889,0.000000,0.000000,3.457778e+02,2.764444e+02,18.206642,12.814729,0.000000,809.000000,137.486884,253.537452,112424.021399,0.000000,0.000000,0.000000,0.777778,0.222222,0.166667,0.000000,0.000000,0.222222,143.633230,50.903690,269.197025,3.457778e+

In [ ]:
# (1) SELECT | ALL features
X = activity_flows_all.iloc[:, 1:(activity_flows_all.shape[1]-1)]
Y = activity_flows_all.iloc[:, -1]


In [ ]:
# create hypergraph
# connection = traffic.groupby([' SrcHost', ' DstHost'], as_index=False)
# connection = connection.agg({' Flow Duration' : 'count'})
src_host_ids = [host_dict[host] for host in basic_flows[' SrcHost']]
dst_host_ids = [host_dict[host] for host in basic_flows[' DstHost']]
edges = list(zip(src_host_ids, dst_host_ids))
hypergraph = {i: edges[i] for i in range(len(edges))}

In [ ]:
# # generate features
features = X
features = np.nan_to_num(features)
features = preprocessing.normalize(features)
features = csr_matrix(features)

# generate features from feature selection set
# X = X.fillna(0)
# selected_features = selectFeatures(X, Y)
# print(f'Selected Features "from All": {selected_features}')
# features = X[selected_features]
# print("Shape of features: ", features.shape)

# features = preprocessing.normalize(features)
# features = csr_matrix(features)


In [ ]:
# generate labels
labels = Y


0         0
1         0
2         0
3         0
4         0
         ..
435761    0
435762    0
435763    0
435764    0
435765    0
Name:  Label, Length: 435766, dtype: int64


In [ ]:
# train/test split

splits = {}
V = len(hosts)
node_sorted = list(range(V))
rnd.shuffle(node_sorted)
train_size = round(0.8*V)
test_size = V - train_size
train_set, test_set = train_test_split(node_sorted, train_size=train_size, test_size=test_size)



splits['train'] = train_set
splits['test'] = test_set


In [ ]:
train_atk_count = 0
for idx, train_id in enumerate(train_set):
    if labels[train_id] !=  0:
      train_atk_count += 1

test_atk_count = 0
for idx, test_id in enumerate(test_set):
    if labels[test_id] != 0:
      test_atk_count += 1

print("Number of atker in train set: ", train_atk_count)
print("Number of atker in test set: ", test_atk_count)

Number of atker in train set:  38800
Number of atker in test set:  9594


In [ ]:
# generate files
with open("hypergraph.pickle", "wb") as hypergraph_out:
    pickle.dump(hypergraph, hypergraph_out)
    hypergraph_out.close()
with open("features.pickle", "wb") as features_out:
    pickle.dump(features, features_out)
    features_out.close()
with open("labels.pickle", "wb") as labels_out:
    pickle.dump(labels, labels_out)
    labels_out.close()
with open("1.pickle", "wb") as splits_out:
    pickle.dump(splits, splits_out)
    splits_out.close()